Inserts all necessary libraries

In [ ]:
import gzip
import requests
import json
import csv
from os import path
import pickle
import csv
import gc
import pandas as pd
import numpy as np
import zipfile

Mount google drive

In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

drive_path = "/content/drive/Shareddrives/KNN-Recommenders/data/"

Mounted at /content/drive


Set dataset variant to download

In [ ]:
# Variant = 0 => all genres | 1 => Poetry | 2 => Comics
variant = 2

# dataset cleaning
# all ... avg 118 interactions per user -> at least 10 for each user
# poetry ... avg 4 interactions per user -> at least 2 for each user
# comics ... avg 30 interactions per user  -> at least 5 for each user
min_interactions = 10 if variant == 0 else 2 if variant == 1 else 5

url_book_metadata = {0:"https://drive.google.com/uc?id=1LXpK1UfqtP89H1tYy0pBGHjYk8IhigUK&confirm=t",
                     1:"https://drive.google.com/uc?id=1H6xUV48D5sa2uSF_BusW-IBJ7PCQZTS1&confirm=t",
                     2:"https://drive.google.com/uc?id=1ICk5x0HXvXDp5Zt54CKPh5qz1HyUIn9m&confirm=t"}

filename_metadata = {0:"goodreads_books.json.gz",
                     1:"goodreads_books_poetry.json.gz",
                     2:"goodreads_books_comics_graphic.json.gz"}

csv_metadata_string = {0:"book_metadata_string_all.csv",
                       1:"book_metadata_string_poetry.csv",
                       2:"book_metadata_string_comics_graphic.csv"}

csv_metadata_int = {0:"book_metadata_int_all.csv",
                    1:"book_metadata_int_poetry.csv",
                    2:"book_metadata_int_comics_graphic.csv"}

url_user_interaction = {0:"https://drive.google.com/u/0/uc?id=1zmylV7XW2dfQVCLeg1LbllfQtHD2KUon&export=download&confirm=t",
                        1:"https://drive.google.com/uc?id=17G5_MeSWuhYnD4fGJMvKRSOlBqCCimxJ&confirm=t",
                        2:"https://drive.google.com/uc?id=1CCj-cQw_mJLMdvF_YYfQ7ibKA-dC_GA2&confirm=t"}

filename_user_interaction = {0:"goodreads_interactions.csv",
                             1:"goodreads_interactions_poetry.json.gz",
                             2:"goodreads_interactions_comics_graphic.json.gz"}

csv_interaction = {0:"book_interactions_all.csv",
                   1:"book_interactions_poetry.csv",
                   2:"book_interactions_comics_graphic.csv"}

Download JSON with author names and IDs

In [ ]:
url_authors = "https://drive.google.com/uc?id=19cdwyXwfXx_HDIgxXaHzH0mrx8nMyLvC"
filename_authors = "authors.json.gz"

if not path.exists(drive_path + filename_authors):
    with open(drive_path + filename_authors, "wb") as f:
        r = requests.get(url_authors)
        f.write(r.content)

Create and save dict {author_id : author_name}

In [ ]:
author_name_id = {}
if not path.exists(drive_path + "author_name_id.pkl"):
    with gzip.GzipFile(drive_path + filename_authors, "rb") as fin:
        for line in fin:
            line_dics = json.loads(line.decode("utf-8"))
            author_name_id[line_dics["author_id"]] = line_dics["name"]
    
    with open(drive_path + "author_name_id.pkl", "wb") as f:
        pickle.dump(author_name_id, f)
else:
    with open(drive_path + "author_name_id.pkl", "rb") as f:
        author_name_id = pickle.load(f)

Book and user number to id mapping

In [ ]:
url_booknumber_to_bookid = "https://drive.google.com/uc?id=1CHTAaNwyzvbi1TR08MJrJ03BxA266Yxr"
filename_booknumber_to_bookid = "booknumber_to_bookid.csv"
if not path.exists(drive_path + filename_booknumber_to_bookid):
    with open(drive_path + filename_booknumber_to_bookid, "wb") as f:
        r = requests.get(url_booknumber_to_bookid)
        f.write(r.content)

In [ ]:
url_user_id_to_number = "https://drive.google.com/uc?id=15ax-h0Oi_Oyee8gY_aAQN6odoijmiz6Q"
filename_user_id_to_number = "user_id_map"
if not path.exists(drive_path + filename_user_id_to_number):
    with open(drive_path + filename_user_id_to_number, "wb") as f:
        r = requests.get(url_user_id_to_number)
        f.write(r.content)

Download and process data of books genres

In [ ]:
url_genres = "https://drive.google.com/uc?id=1ah0_KpUterVi-AHxJ03iKD6O0NfbK0md"
filename_genres = "gooreads_book_genres_initial.json.gz"

if not path.exists(drive_path + filename_genres):
    with open(drive_path + filename_genres, "wb") as f:
        r = requests.get(url_genres)
        f.write(r.content)

books_genres = {}
if not path.exists(drive_path + "books_genres.pkl"):
    with gzip.GzipFile(drive_path + filename_genres, "rb") as f:
        for line in f:
            line_dics = json.loads(line.decode("utf-8"))
            genres_str = ""            
            # list -> string
            for g in list(line_dics["genres"].keys()):
                genres_str = genres_str + " " + g
                books_genres[line_dics["book_id"]] = genres_str
    
    with open(drive_path + "books_genres.pkl", "wb") as f:
        pickle.dump(books_genres, f)

else:
    with open(drive_path + "books_genres.pkl", "rb") as f:
        books_genres = pickle.load(f)

Download and process interactions. Drop interactions with rating 0. Remove books with few interactions.

In [ ]:
if not path.exists(drive_path + filename_user_interaction[variant]):
    with open(drive_path + filename_user_interaction[variant], "wb") as f:
        r = requests.get(url_user_interaction[variant])
        f.write(r.content)

if variant:
    book_interactions = []
    user_id_map = pd.read_csv(drive_path + "user_id_map", names=["user_id_csv", "user_id"], index_col = 1, low_memory=False)
    with gzip.GzipFile(drive_path + filename_user_interaction[variant], "rb") as f:
        for line in f:
            l = json.loads(line.decode("utf-8"))
            if l["rating"] != 0: # we want only explicit interactions -> drop those with rating 0
                book_interactions.append([user_id_map.loc[l["user_id"]].user_id_csv, l["book_id"], l["rating"]])

    df_interactions = pd.DataFrame(book_interactions, columns=["user_id", "book_id", "rating"])
else:
    df_interactions = pd.read_csv(drive_path + filename_user_interaction[variant], usecols=["user_id", "book_id", "rating"], low_memory=True)
    df_interactions = df_interactions[df_interactions.rating != 0] # we want only explicit interactions -> drop those with rating 0

# remove interactions where books/users don't have enough
old_count = 0
while df_interactions.shape[0] != old_count:
    old_count = df_interactions.shape[0]
    grouped_books = df_interactions.groupby(by="book_id").agg({"user_id":"count"}).rename(columns={"user_id":"review_count"})
    wanted_books = grouped_books[grouped_books.review_count >= min_interactions]
    grouped_users = df_interactions.groupby(by="user_id").agg({"book_id":"count"}).rename(columns={"book_id":"review_count"})
    wanted_users = grouped_users[grouped_users.review_count >= min_interactions]
    df_interactions = df_interactions[df_interactions["book_id"].isin(wanted_books.index) & df_interactions["user_id"].isin(wanted_users.index)]

df_interactions.sort_values(by="user_id") 

df_interactions.to_csv(drive_path + csv_interaction[variant], header=["user_id", "book_id", "rating"], index=False)

Download and process books metadata. We only use some columns. 


In [ ]:
if not path.exists(drive_path + filename_metadata[variant]):
    with open(drive_path + filename_metadata[variant], "wb") as f:
        r = requests.get(url_book_metadata[variant])
        f.write(r.content)

book_metadata_string = []
book_metadata_int = []
with gzip.GzipFile(drive_path + filename_metadata[variant], "rb") as f:   
    book_ids = df_interactions["book_id"].unique()
    for line in f:
        authors = series = ""
        l = json.loads(line.decode("utf-8"))
        book_id = l["book_id"]
        #if book doesnt have interaction, continue
        if book_id not in book_ids:
            continue

        #Get all authors with empty role
        for author in l["authors"]:
            if author["role"] == '':
                authors = authors + " " + str(author_name_id[author["author_id"]])

        #concat all series 
        for serie in l["series"]:
            series = series + " " + str(serie)
        

        book_metadata_string.append([str(book_id),
                                     l["title"] + " " +
                                     l["publisher"] + " " + 
                                     l["description"] + " " +
                                     authors + " " +
                                     series + " " +
                                     books_genres[book_id]])

        num_pages = l["num_pages"] if l["num_pages"].isnumeric() else -1
        text_reviews_count = l["text_reviews_count"] if l["text_reviews_count"].isnumeric() else -1
        ratings_count = l["ratings_count"] if l["ratings_count"].isnumeric() else -1
        average_rating = l["average_rating"];
        try:
            float(average_rating)
        except:
            average_rating = -1.0

        book_metadata_int.append([int(book_id),
                                  int(ratings_count),
                                  int(num_pages),
                                  float(average_rating),
                                  int(text_reviews_count)])

with open(drive_path + csv_metadata_string[variant], "w") as f:
    writer = csv.writer(f, delimiter=",")
    writer.writerow(["book_id", "aggregated_string"])
    writer.writerows(book_metadata_string)

with open(drive_path + csv_metadata_int[variant], "w") as f:
    writer = csv.writer(f, delimiter=",")
    writer.writerow(["book_id", "ratings_count", "num_pages", "average_rating", "text_reviews_count"])
    writer.writerows(book_metadata_int)

User-book interactions

In [ ]:
pd.read_csv(drive_path + csv_interaction[variant])

Int vectors

In [ ]:
pd.read_csv(drive_path + csv_metadata_int[variant])

String vectors

In [ ]:
pd.read_csv(drive_path + csv_metadata_string[variant])

Get Kaggle 10k dataset for testing purposes

In [ ]:
! pip install kaggle
! mkdir ~/.kaggle
! chmod 600 /content/drive/Shareddrives/KNN-Recommenders/kaggle.json
! cp /content/drive/Shareddrives/KNN-Recommenders/kaggle.json ~/.kaggle/kaggle.json

In [ ]:
goodbooks_filename = drive_path + "goodbooks-10k.zip"
if not path.exists(goodbooks_filename):
    ! kaggle datasets download -d zygmunt/goodbooks-10k -p "/content/drive/Shareddrives/KNN-Recommenders/data/"

In [ ]:
zf = zipfile.ZipFile(goodbooks_filename)
df_books = pd.read_csv(zf.open("books.csv"), usecols=["id", "book_id"])
df_interactions = pd.read_csv(zf.open("ratings.csv"))
# it's obsolete dataset - has duplicates (updated doesn't have so many references to compare + is much bigger)
x = df_interactions[["book_id", "user_id"]].drop_duplicates().index
df_interactions= df_interactions.loc[x, :]

# kaggle ratings hav bookids 1-10000 - not real book ids - change
id_to_book_id = {row["id"]:row["book_id"] for _, row in df_books.iterrows()}
df_interactions["book_id"] = [id_to_book_id[id] for id in df_interactions["book_id"]]

# filter existing books by kaggle books - presumes existing all books metadata files
df_books = df_books.drop(columns="id")
metadata_int_df = pd.read_csv(drive_path + csv_metadata_int[0])
metadata_int_df = df_books.merge(metadata_int_df, on=['book_id'])
metadata_string_df = pd.read_csv(drive_path + csv_metadata_string[0])
metadata_string_df = df_books.merge(metadata_string_df, on=['book_id'])

# some books might have been filtered out
df_interactions = df_interactions[df_interactions["book_id"].isin(metadata_int_df["book_id"])]

# save
df_interactions.to_csv(drive_path + "book_interactions_kaggle.csv", index=False)
metadata_int_df.to_csv(drive_path + "book_metadata_int_kaggle.csv", index=False)
metadata_string_df.to_csv(drive_path + "book_metadata_string_kaggle.csv", index=False)